In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from datetime import datetime as dt

from xgboost import XGBClassifier
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 

from catboost import CatBoostClassifier

# Read Data

In [2]:
df_credits = pd.read_csv('../data/User Credits Student Access.csv', encoding='utf-8')
df_atlas = pd.read_csv('../data/Atlas Cechu Student Access.csv', encoding='utf-8')
df_payments = pd.read_csv('../data/Payments Student Access.csv', encoding='utf-8')

In [3]:
df_payments[(df_payments['user'].isna())&(df_payments['credits']>=500)]
df_payments[(df_payments['user'].isna())&(df_payments['credits']<500)]
df_payments[(df_payments['user'].isna())&(df_payments['credits'].isna())]

df_payments[df_payments['id'].isnull()]

,id,created_at,changed_at,user,batch,credits,state,type
1424,NaN,Anna,Prchalová,NaN,1299589028/3030,NaN,NaN,NaN


In [4]:
df_atlas.columns

Index(['user_id', 'all_intro-2022', 'atlas_vzorek-2023', 'atlas_vzorek-2024',
       'all_intro-trendaro_panel_all', 'sex-woman', 'sex-man', 'age-15_17',
       'age-18_24', 'age-25_34',
       ...
       'check-tv', 'check-radio', 'check-household', 'check-overdraft',
       'check-naramek', 'isic2_age-21_26', 'isic2_age-15_20', 'isic_age-15_17',
       'isic_age-18_22', 'isic_age-23_26'],
      dtype='object', length=1121)

In [80]:
df_credits

,user,credits,is_active,is_verified,is_locked,wage
0,STUD66006,25,1,1,0,Peníze
1,STUD22095,51,1,1,0,Peníze
2,STUD77411,0,1,1,1,Peníze
3,STUD56329,0,1,0,0,NaN
4,STUD23516,30,1,1,0,Peníze
...,...,...,...,...,...,...
89906,STUD54678,789,1,1,0,Peníze
89907,STUD43709,0,0,1,0,NaN
89908,STUD21881,0,0,0,0,NaN
89909,STUD72037,16,1,1,0,Lékaři bez hranic


# Negative Values investigation & Chekcing other things

In [5]:
df_c_negative = df_credits[df_credits['credits']<0]
df_credits[df_credits.user.isin(df_c_negative.user) == True]

# IDK what I'm doing with payments
df_payments[df_payments.user.isin(df_c_negative.user)==True].sort_values(['user','created_at'])
df_payments.state.unique()
df_payments[df_payments.user=='STUD54678']
df_credits[df_credits.user=='STUD54678']

,user,credits,is_active,is_verified,is_locked,wage
89906,STUD54678,789,1,1,0,Peníze


In [6]:
df_c_negative # from df_credits

,user,credits,is_active,is_verified,is_locked,wage
15877,STUD26214,-416,1,1,0,Peníze
28959,STUD16141,-10,1,1,0,Peníze
71540,STUD91415,-452,1,1,0,Peníze
72659,STUD99479,-441,1,1,0,Peníze
73458,STUD10440,-4,1,1,0,Peníze


In [7]:
df_payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30087 entries, 0 to 30086
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          30086 non-null  float64
 1   created_at  30087 non-null  object 
 2   changed_at  30087 non-null  object 
 3   user        27652 non-null  object 
 4   batch       27502 non-null  object 
 5   credits     30086 non-null  float64
 6   state       30086 non-null  object 
 7   type        30086 non-null  object 
dtypes: float64(2), object(6)
memory usage: 1.8+ MB


# Data Cleaning

In [8]:
##### In the dataset we found out that in payments dataset values that are being cancelled has creadits = 0 and NaN user having credit 430 and still being PAID. However, that user doesn't have user_id

df_payments[df_payments['credits']<500].groupby('state').size()

state
CANCELLED    2000
PAID            1
dtype: int64

In [9]:
df_credits_cleaned = df_credits[df_credits['credits']>0]


"""
df_payments_cleaned = df_payments[(df_payments['user'].isna())&(df_payments['credits']<500)]
df_payments_cleaned = df_payments[(df_payments['user'].isna())&(df_payments['credits']>=500)]
df_payments_cleaned[(df_payments['user'].isna())&(df_payments['credits'].isna())]

-> everything cane be done using: df_payments_cleaned = df_payments[df_payments['user'].notna()]

"""

df_payments_cleaned = df_payments[df_payments['user'].notna()] # we threw out from payments 2345 observations


In [10]:
df_payments_cleaned

,id,created_at,changed_at,user,batch,credits,state,type
0,30116.0,2025-04-07 10:32:05.073604+00:00,2025-04-07 10:32:05.073633+00:00,STUD83031,NaN,509.0,REQUEST,MONEY
1,30115.0,2025-04-07 10:30:09.304166+00:00,2025-04-07 10:30:09.304196+00:00,STUD356,NaN,538.0,REQUEST,MONEY
2,30114.0,2025-04-07 09:57:04.343935+00:00,2025-04-07 09:57:04.343985+00:00,STUD25955,NaN,542.0,REQUEST,MONEY
3,30113.0,2025-04-07 09:15:39.069868+00:00,2025-04-07 09:15:39.069910+00:00,STUD45618,NaN,501.0,REQUEST,MONEY
4,30112.0,2025-04-07 08:28:47.838506+00:00,2025-04-07 08:28:47.838578+00:00,STUD44458,NaN,504.0,REQUEST,MONEY
...,...,...,...,...,...,...,...,...
30080,36.0,2017-08-11 14:55:18.396493+00:00,2024-01-10 17:31:11.782802+00:00,STUD49275,5,506.0,PAID,MONEY
30081,35.0,2017-08-11 14:54:35.456650+00:00,2024-01-10 17:31:11.801715+00:00,STUD53465,5,523.0,PAID,MONEY
30082,34.0,2017-08-11 14:06:49.193910+00:00,2024-01-10 17:31:11.820777+00:00,STUD29324,5,511.0,PAID,MONEY
30085,31.0,2017-08-10 21:58:35.586238+00:00,2024-01-10 17:31:11.867765+00:00,STUD92308,NaN,511.0,PAID,MEDICINS_SANS_FRONTIERS


# What is core table for merge? - Payments // Can't I just use inner join? Therefore I dont' have to work with nul values - I don't want them in model anyway

In [11]:
#I have an assumption that all users should be in Atlas Cechu. However, after cheking my assumption it tourns out not to be true. 

# Convert the boolean series to a DataFrame with a column name
atlas_in_credits = df_credits_cleaned.user.isin(df_atlas.user_id).rename('User in Atlas')
atlas_in_payments = df_payments_cleaned.user.isin(df_atlas.user_id)

# Plotting
fig_ac = px.bar(atlas_in_credits.value_counts(), title='How many users from Atlas Cechu are in the credits table', labels={'index': 'User in Atlas (True/False)', 'value': 'Count'})
fig_ap = px.bar(atlas_in_payments.value_counts(), title='How many users from Atlas Cechu are in the payments table', labels={'index': 'User in Atlas (True/False)', 'value': 'Count'})

fig_ac.show()
fig_ap.show()

In [12]:
df_atlas.user_id.isin(df_credits_cleaned.user).value_counts()

user_id
True     40672
False     2861
Name: count, dtype: int64

In [13]:
df_atlas.user_id.isin(df_payments_cleaned.user).value_counts()

user_id
False    34824
True      8709
Name: count, dtype: int64

Since we want to find out when who pulls what, it makes sense to use the payments table and connect data to it - even though there are users in payments tables that are not in Atlas Cechu

In [88]:
#df_merge_credits = pd.merge(df_payments_cleaned, df_credits_cleaned, how="inner", left_on='user', right_on='user')
#df_merge_credits[df_merge_credits.user=='STUD92308']

"""
I don't find credits table to be interesting - mainly we cannot join them together due to different "time type" -- credits shows the newest version of creadits without historical changes, on the other hand table payments shows history of all transactions
"""

'\nI don\'t find credits table to be interesting - mainly we cannot join them together due to different "time type" -- credits shows the newest version of creadits without historical changes, on the other hand table payments shows history of all transactions\n'

In [14]:
df_merge_full = pd.merge(df_payments_cleaned, df_atlas, how='inner', left_on='user', right_on='user_id')
df_merge_full.rename(columns={'credits_x':'credits_payments', 'credits_y':'credits_credits'})
df_merge_full = pd.get_dummies(df_merge_full, columns=['type'], drop_first=True)
df_merge_full = df_merge_full.astype({col: int for col in df_merge_full.select_dtypes(include='bool').columns})


# df_merge_full.to_csv('../data_output/df_merge_full.csv')


df_merge_full['created_at'] = pd.to_datetime(df_merge_full['created_at'])

df_merge_full['day'] = df_merge_full['created_at'].dt.day
df_merge_full['month'] = df_merge_full['created_at'].dt.month
df_merge_full['year'] = df_merge_full['created_at'].dt.year
df_merge_full['hour'] = df_merge_full['created_at'].dt.hour
df_merge_full['weekday'] = df_merge_full['created_at'].dt.weekday


df_merge_full = df_merge_full.drop(columns=['created_at'])

/var/folders/_c/8b17td616wg63_3tyjv1dhb80000gn/T/ipykernel_50764/2234070991.py:12: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/_c/8b17td616wg63_3tyjv1dhb80000gn/T/ipykernel_50764/2234070991.py:13: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/_c/8b17td616wg63_3tyjv1dhb80000gn/T/ipykernel_50764/2234070991.py:14: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd

In [15]:
df_merge_full

,id,changed_at,user,batch,credits,state,user_id,all_intro-2022,atlas_vzorek-2023,atlas_vzorek-2024,...,isic_age-23_26,type_HANDIPET,type_MEDICINS_SANS_FRONTIERS,type_MONEY,type_TRENDARO,day,month,year,hour,weekday
0,30116.0,2025-04-07 10:32:05.073633+00:00,STUD83031,NaN,509.0,REQUEST,STUD83031,1,0,0,...,0,0,0,1,0,7,4,2025,10,0
1,30115.0,2025-04-07 10:30:09.304196+00:00,STUD356,NaN,538.0,REQUEST,STUD356,1,0,1,...,0,0,0,1,0,7,4,2025,10,0
2,30114.0,2025-04-07 09:57:04.343985+00:00,STUD25955,NaN,542.0,REQUEST,STUD25955,1,0,0,...,0,0,0,1,0,7,4,2025,9,0
3,30113.0,2025-04-07 09:15:39.069910+00:00,STUD45618,NaN,501.0,REQUEST,STUD45618,1,0,0,...,0,0,0,1,0,7,4,2025,9,0
4,30112.0,2025-04-07 08:28:47.838578+00:00,STUD44458,NaN,504.0,REQUEST,STUD44458,1,0,0,...,0,0,0,1,0,7,4,2025,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26874,36.0,2024-01-10 17:31:11.782802+00:00,STUD49275,5,506.0,PAID,STUD49275,1,0,0,...,0,0,0,1,0,11,8,2017,14,4
26875,35.0,2024-01-10 17:31:11.801715+00:00,STUD53465,5,523.0,PAID,STUD53465,1,0,0,...,0,0,0,1,0,11,8,2017,14,4
26876,34.0,2024-01-10 17:31:11.820777+00:00,STUD29324,5,511.0,PAID,STUD29324,1,0,1,...,0,0,0,1,0,11,8,2017,14,4
26877,31.0,2024-01-10 17:31:11.867765+00:00,STUD92308,NaN,511.0,PAID,STUD92308,1,1,0,...,0,0,1,0,0,10,8,2017,21,3


In [16]:
# split dataset into three categories

df_modeling = df_merge_full.drop(columns=['id', 'changed_at', 'user', 'batch', 'state', 'user_id'])
df_modeling['credits_601+'] = (df_modeling['credits'] > 600).astype(int)
df_modeling = df_modeling.drop(columns=['credits'])

fig = px.bar(pd.DataFrame(df_modeling['credits_601+'].value_counts().reset_index()), x='credits_601+', y='count', title='Credits Category Distribution')
fig.show()

/var/folders/_c/8b17td616wg63_3tyjv1dhb80000gn/T/ipykernel_50764/1893180649.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



In [17]:
df_modeling['credits_601+'].unique()

array([0, 1])

In [20]:
df_modeling = df_modeling[df_modeling['credits_601+'].isnull()==False] # clean data from null values

# Modeling

In [21]:
y = df_modeling['credits_601+']
X = df_modeling.drop(columns=['credits_601+'])

def split_data(X, y, test_size=0.2, stratify=True, random_state=42):
    """
    Split data into training and testing sets with optional stratification.

    Args:
        X (pd.DataFrame): Feature matrix.
        y (pd.Series): Target variable.
        test_size (float): Proportion of data to use as test set.
        stratify (bool): Whether to stratify split based on y.
        random_state (int): Seed for reproducibility.

    Returns:
        Tuple: (X_train, X_test, y_train, y_test)
    """
    return train_test_split(
        X,
        y,
        test_size=test_size,
        stratify=y if stratify else None,
        random_state=random_state
    )



X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y , train_size=0.8, random_state=42)

# Grid Search

In [24]:
range(1, 20, 1)

range(1, 20)

In [26]:
# catboost clf

grid_cat = {
    'depth':np.array(range(1, 20, 1)),
    'learning_rate': np.array(range(5, 25, 5))/100,
    'loss_function':['CrossEntropy','Logloss'],
}

model_cat = CatBoostClassifier(
    iterations=500,
    eval_metric='Accuracy',
    verbose=100 # control how often the model will print out output - e.g. when verbose is set to 50 = every 50 iteretion print out current status
)

grid_cat = RandomizedSearchCV(estimator=model_cat, param_distributions=grid_cat, cv=3, scoring='accuracy', n_iter=15)
grid_cat.fit(X_train, y_train)

#model.fit(X_train, y_train, eval_set=(X_test, y_test))
print(f'best params: {grid_cat.best_params_}')
print(f'best accuracy: {grid_cat.best_score_}')

0:	learn: 0.7880712	total: 1.4s	remaining: 11m 39s
100:	learn: 0.9771189	total: 1m 47s	remaining: 7m 4s
200:	learn: 0.9988141	total: 3m 34s	remaining: 5m 18s
300:	learn: 1.0000000	total: 5m 22s	remaining: 3m 33s
400:	learn: 1.0000000	total: 7m 6s	remaining: 1m 45s
499:	learn: 1.0000000	total: 8m 46s	remaining: 0us
0:	learn: 0.7750262	total: 1.01s	remaining: 8m 25s
100:	learn: 0.9761423	total: 1m 40s	remaining: 6m 35s
200:	learn: 0.9995814	total: 3m 21s	remaining: 4m 59s
300:	learn: 1.0000000	total: 5m	remaining: 3m 18s
400:	learn: 1.0000000	total: 6m 40s	remaining: 1m 38s
499:	learn: 1.0000000	total: 8m 18s	remaining: 0us
0:	learn: 0.7638114	total: 9.61ms	remaining: 4.8s
100:	learn: 0.9777483	total: 9.61ms	remaining: 4.8s
200:	learn: 0.9995117	total: 9.61ms	remaining: 4.8s
300:	learn: 0.9999302	total: 9.61ms	remaining: 4.8s
400:	learn: 0.9999302	total: 12.2s	remaining: 1m 26s
499:	learn: 0.9999302	total: 1m 44s	remaining: 0us
0:	learn: 0.7642135	total: 12.3ms	remaining: 6.13s
100:	lear

/Users/vanhieuvu/Documents/school/Magisterský studium/2.semestr/datovy_projekt/dp_env/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning:


9 fits failed out of a total of 45.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
9 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/vanhieuvu/Documents/school/Magisterský studium/2.semestr/datovy_projekt/dp_env/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/vanhieuvu/Documents/school/Magisterský studium/2.semestr/datovy_projekt/dp_env/lib/python3.9/site-packages/catboost/core.py", line 5245, in fit
    self._

0:	learn: 0.7642655	total: 13.9ms	remaining: 6.94s
100:	learn: 0.7816119	total: 1.4s	remaining: 5.53s
200:	learn: 0.8228154	total: 2.7s	remaining: 4.02s
300:	learn: 0.8495559	total: 4.04s	remaining: 2.67s
400:	learn: 0.8731805	total: 5.34s	remaining: 1.32s
499:	learn: 0.8911780	total: 6.64s	remaining: 0us
best params: {'loss_function': 'Logloss', 'learning_rate': np.float64(0.1), 'depth': np.int64(6)}
best accuracy: 0.7901688322117079


In [ ]:
# CatBoost with Cross Validation
from sklearn.model_selection import cross_val_score

model_cat = CatBoostClassifier(
    iterations=1000,
    eval_metric='Accuracy',
    verbose=200,
    depth=6,
    learning_rate=0.1,
    loss_function='Logloss'
)
model_cat.fit(X_train, y_train, eval_set=(X_test, y_test), use_best_model=True)
cv_scores = cross_val_score(model_cat, X_train, y_train, cv=5, verbose=1)
print(f'CV Scores: {cv_scores}')
print(f'Mean CV Score: {np.mean(cv_scores)}')

0:	learn: 0.7642655	test: 0.7635789	best: 0.7635789 (0)	total: 14.2ms	remaining: 14.2s
200:	learn: 0.8228154	test: 0.7834821	best: 0.7834821 (199)	total: 2.83s	remaining: 11.3s
400:	learn: 0.8731805	test: 0.7937128	best: 0.7955729 (346)	total: 5.54s	remaining: 8.28s
600:	learn: 0.9088964	test: 0.8000372	best: 0.8002232 (596)	total: 8.3s	remaining: 5.51s
800:	learn: 0.9361019	test: 0.8004092	best: 0.8037574 (730)	total: 11s	remaining: 2.74s
999:	learn: 0.9560061	test: 0.7989211	best: 0.8037574 (730)	total: 13.8s	remaining: 0us

bestTest = 0.8037574405
bestIteration = 730

Shrink model to first 731 iterations.
0:	learn: 0.7639809	total: 12.6ms	remaining: 12.6s
200:	learn: 0.8279851	total: 3s	remaining: 11.9s
400:	learn: 0.8834438	total: 5.91s	remaining: 8.83s
600:	learn: 0.9244274	total: 8.86s	remaining: 5.88s
800:	learn: 0.9537263	total: 11.9s	remaining: 2.96s
999:	learn: 0.9744797	total: 14.9s	remaining: 0us
0:	learn: 0.7639228	total: 8.8ms	remaining: 8.79s
200:	learn: 0.8256598	total:

In [38]:
grid_cat.best_score_, grid_cat.best_params_

(np.float64(0.7901688322117079),
 {'loss_function': 'Logloss',
  'learning_rate': np.float64(0.1),
  'depth': np.int64(6)})

In [27]:
# XGBClassifier

xgb.set_config(verbosity=1)

param_grid = {
    'n_estimators': np.array(range(100, 2000, 100)),
    'max_depth': np.array(range(1, 20, 1)),
    'learning_rate': np.array(range(5, 25, 5))/100,
    'eval_metric': ['logloss', 'error', 'auc'],
}

model = XGBClassifier(objective='binary:logistic', use_label_encoder=False)

grid_search = RandomizedSearchCV(model, param_grid, cv=3, scoring='accuracy', n_iter=20)
grid_search.fit(X_train, y_train)

print(f'Best Parameters: {grid_search.best_params_}')
print(f'Best Score: {grid_search.best_score_:.4f}')

# ✅ Predict and evaluate using the best model
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

/Users/vanhieuvu/Documents/school/Magisterský studium/2.semestr/datovy_projekt/dp_env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning:

[18:24:38] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.


/Users/vanhieuvu/Documents/school/Magisterský studium/2.semestr/datovy_projekt/dp_env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning:

[18:24:56] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.


/Users/vanhieuvu/Documents/school/Magisterský studium/2.semestr/datovy_projekt/dp_env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning:

[18:25:14] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.


/Users/vanhieuvu/Documents/school/Magisterský studium/2.semestr/datovy_projekt/dp_env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning:

[18:25:32] WARNING: 

Best Parameters: {'n_estimators': np.int64(1300), 'max_depth': np.int64(3), 'learning_rate': np.float64(0.1), 'eval_metric': 'auc'}
Best Score: 0.7902
              precision    recall  f1-score   support

           0       0.82      0.94      0.88      4106
           1       0.64      0.32      0.43      1270

    accuracy                           0.80      5376
   macro avg       0.73      0.63      0.65      5376
weighted avg       0.78      0.80      0.77      5376

Accuracy: 0.7978050595238095


In [210]:
# logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluate
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

#Accuracy: 0.7457324335053592

              precision    recall  f1-score   support

           0       0.76      0.97      0.85      3768
           1       0.48      0.09      0.15      1270

    accuracy                           0.75      5038
   macro avg       0.62      0.53      0.50      5038
weighted avg       0.69      0.75      0.67      5038

Accuracy: 0.7457324335053592


/Users/vanhieuvu/Documents/school/Magisterský studium/2.semestr/datovy_projekt/dp_env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



In [28]:

# Define parameter grid
param_grid = {
    'max_depth': np.array(range(1, 20, 1)),
    'criterion': ['gini', 'entropy']
}

# Define the base model
model_dt = DecisionTreeClassifier(random_state=42)

# Run grid search
grid_search = RandomizedSearchCV(model_dt, param_grid, cv=3, scoring='accuracy', n_iter=20)
grid_search.fit(X_train, y_train)

# Get best model and predict
best_dt = grid_search.best_estimator_
y_pred = best_dt.predict(X_test)

print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      1.00      0.87      4106
           1       0.00      0.00      0.00      1270

    accuracy                           0.76      5376
   macro avg       0.38      0.50      0.43      5376
weighted avg       0.58      0.76      0.66      5376

Accuracy: 0.7637648809523809


/Users/vanhieuvu/Documents/school/Magisterský studium/2.semestr/datovy_projekt/dp_env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/vanhieuvu/Documents/school/Magisterský studium/2.semestr/datovy_projekt/dp_env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/vanhieuvu/Documents/school/Magisterský studium/2.semestr/datovy_projekt/dp_env/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [29]:
# Random Forest
# Define the parameter grid
param_grid = {
    'n_estimators': np.array(range(100, 2000, 100)),
    'max_depth': np.array(range(1, 20, 1)),
    'criterion': ['gini', 'entropy']
}

# Base model
model_rf = RandomForestClassifier(random_state=42)

# Grid search
grid_search = RandomizedSearchCV(model_rf, param_grid, cv=3, scoring='accuracy', n_iter=20)
grid_search.fit(X_train, y_train)

# Use the best model
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test)

# Evaluate
print(classification_report(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Best Parameters:", grid_search.best_params_)

              precision    recall  f1-score   support

           0       0.82      0.94      0.88      4106
           1       0.63      0.32      0.43      1270

    accuracy                           0.80      5376
   macro avg       0.73      0.63      0.65      5376
weighted avg       0.77      0.80      0.77      5376

Accuracy: 0.7961309523809523
Best Parameters: {'n_estimators': np.int64(900), 'max_depth': np.int64(16), 'criterion': 'entropy'}
